In [74]:
import os
import sqlite3
#import appscript
from appscript import *

In [75]:
#to get ssl working
#pip install ndg-httpsclient
#pip install pyopenssl
#pip install pyasn1
#unset https_proxy

In [76]:
#parse email and save to dataframe

In [77]:
import imaplib
import pandas as pd
import email
from bs4 import BeautifulSoup



mail = imaplib.IMAP4_SSL('imap.gmail.com', 993)
mail.login('lodge.brennan@gmail.com', 'password')
#mail.select('"[Gmail]/tub06054@temple.edu"')
mail.select('"tub06054@temple.edu"')
result, data = mail.search(None, '(SINCE "01-Jan-2012")')
uids = data[0].split()





In [78]:
#Loop through each uid to fetch headers
import sqlite3 as db
import re
msg_dict = None
newdata = pd.DataFrame()
for uid in uids:
    if result == 'OK':
    
        resp, data = mail.fetch(uid, "(RFC822)")
        
        full_message = ""
        msg_dict = None
        df = pd.DataFrame()
        
        body_base64 = ""

        raw_email = data[0][1] # here's the body, which is raw text of the whole email

        # including headers and alternate payloads
              
        #msg = email.message_from_bytes(raw_email)
        
        msg = email.message_from_string(str(raw_email, 'utf-8'))
        #print(msg)

        msg_items = msg.items()
        msg_dict = dict(msg_items)

                
        body_part = {}

        for part in msg.walk():
            body_part = {
                #"Message-ID": key,
                "MESSAGE_TYPE": str(part.get_content_maintype()),
                "CHARSETS": str(part.get_charsets()),
                "DEFULAT_TYPE:": str(part.get_default_type()),
                "CONTENT_CHARSET:": str(part.get_content_charset())}

        body = {}
        text = ""
        '''
        check = ""
        parser = email.parser.Parser()
        parts = msg.get_payload()
        check = parts[0].get_content_type()
        if msg.is_multipart():
            if check == "text/plain":
                text = parts[0].get_payload()

            #print parts[0].get_payload()
            elif check == "multipart/alternative":
                part = parts[0].get_payload()

            #print parts[0].get_payload()
                if part[0].get_content_type() == "text/plain":
                    try:
                        text = part[0].get_payload()
                    except:
                        pass
        '''
        if msg.is_multipart():
            for part in msg.walk():

                try:
                    text = part.get_payload(decode=True)
                    missing_padding = 4 - len(text) % 4

                    if missing_padding:
                        text += b'='* missing_padding

                        text += base64.urlsafe_b64decode(text)
                        #print body_base64 + "&"*80
                    else:
                        text += base64.urlsafe_b64decode(body)
                        #print body_base64 + "&"*80

                except:
                    pass
            else:
                if text is None:
                        charset = str(msg.get_content_charset())
                        text = unicode(msg.get_payload(decode=True),charset, "ignore").encode('utf8','ignore')
                        text = text.strip()
            '''
            #if text is not None:
                #text = text.strip()
            #else:
                #html = html.strip()

            '''
        #text = BeautifulSoup(text.decode('utf-16', 'ignore'))
        #remove uncessary spaces
        def removeinvalid(y):
            y=''.join([x for x in y if ord(x) < 128])
            return y
        soup = BeautifulSoup(text,features='html.parser')
        text = re.sub(r'[\n\r\t]{2,}', ' ', soup.get_text(separator=u"\n"))
        text = removeinvalid(str(text))
        text = text.encode('utf-8')
        msg_dict['body_part'] = body_part
        msg_dict['body'] = text
        msg_dict = {k:[v] for k,v in msg_dict.items()}
        
        #append new msg dict rows to dataframe 
        df_new = pd.DataFrame([msg_dict])
        df_new.columns = df_new.columns.str.lower()
        #drop duplicate columns 
        df_new = df_new.loc[:, ~df_new.columns.duplicated()]
        newdata = pd.concat([newdata, df_new])    
        #convert to dataframe and ensure all values are strings for sqlite
        newdata = pd.DataFrame(newdata)
        newdata = newdata.applymap(str)
        
        
    else:
        pass


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


In [79]:
newdata.columns

Index(['delivered-to', 'received', 'domainkey-status', 'received-spf',
       'x-gmail-fetch-info', 'return-path', 'authentication-results', 'x-sbrs',
       'dkim-signature', 'domainkey-signature', 'date', 'from', 'reply-to',
       'to', 'message-id', 'subject', 'mime-version', 'content-type', 'x-mid',
       'x-orgid', 'list-unsubscribe', 'body_part', 'body',
       'content-transfer-encoding', 'x-originating-ip', 'x-mailer', 'sender',
       'x-linkedin-template', 'x-linkedin-class', 'x-linkedin-fbl',
       'x-mailer-lid', 'x-mailer-recptid', 'x-mailer-sid', 'x-mailer-sent-by',
       'content-disposition', 'x-antiabuse', 'x-dkim', 'x-sgxh1', 'x-rext',
       'x-cid', 'x-priority', 'errors-to', 'x-list-unsubscribe',
       'x-unsubscribe-web', 'x-icpinfo', 'x-return-path-hint', 'x-eid',
       'x-pid', 'x-virtualserver', 'x-mailingid', 'x-smheadermap',
       'x-destination-id', 'x-smfbl', 'x-virtualservergroup', 'guid',
       'x-indid', 'x-vendor', 'x-campaignid', 'x-mailer-vers

In [80]:
#add raw emails to sqlite db named ham_emails.db with table name TUmain01012012

In [81]:
#cleanse script 
from pandas import DataFrame, Series
import numpy as np
dataframe_ham = newdata
length = len(dataframe_ham)

feature_first_time_from = DataFrame(dataframe_ham.duplicated(['from']))

feature_first_time_from.columns = ['feature_first_time_from']
features_ham = pd.concat([dataframe_ham, feature_first_time_from], axis=1)

#Links in body
bodylink = dataframe_ham['body']
#bodylink = str(bodylink)
url_pattern = r'(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]{0,255}[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]?|(\([^\s()<>]?\)))?\))?(?:\(([^\s()<>]?|(\([^\s()<>]?\)))?\)|[^\s`!()\[\]{};:\'".,<>?  \xab\xbb\u201c\u201d\u2018\u2019]))'

extracted_data = bodylink.str.extract(url_pattern)
body_url = DataFrame(extracted_data[0])

#counts of body_url links
body_url.columns = ['feature_body_url']
body_url = DataFrame(body_url['feature_body_url'])
#body_url['feature_body_url'].value_counts()

# #Subject Charset Type:
subject = dataframe_ham['subject']
subject = DataFrame(subject)

# #ContentType_body
ContentType_df = DataFrame(dataframe_ham['content-type'])
ContentType_df = ContentType_df['content-type'].str.split(';').str[0]


features_ham = pd.concat([features_ham, ContentType_df], axis=1)
##IP
##pattern = re.compile(r'(\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3})')

recieved_spf = dataframe_ham['received-spf']
ip_pattern = r'(?<=client-ip=)([0-9\.]*)(?=;)'
ip_address = recieved_spf.str.extract(ip_pattern)

#ip_address to dataFrame
ip_of_sender = DataFrame(ip_address)
#ip_of_sender = DataFrame(ip_of_sender['received-spf'])
ip_of_sender.columns = ['ip_of_sender']
ip_of_sender = DataFrame(ip_of_sender)
#ip_of_sender['Received-SPF'].isnull().value_counts()
#ip_of_sender = ip_of_sender.reset_index()

features_ham = pd.concat([features_ham, ip_of_sender], axis=1)


#LINK DataFrame
def regex_link(text):
    link = str(re.findall(r'(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]{0,255}[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]?|(\([^\s()<>]?\)))?\))?(?:\(([^\s()<>]?|(\([^\s()<>]?\)))?\)|[^\s`!()\[\]{};:\'".,<>?  \xab\xbb\u201c\u201d\u2018\u2019]))', text))
    link.encode('utf-8').strip()
    if len(link) == 0:
        link = "None"
    return link
    
link_df = dataframe_ham['body'].apply(regex_link)
link_df = pd.DataFrame(link_df)
link_df = link_df.rename(columns={'body': 'links'})

features_ham = pd.concat([features_ham, link_df], axis=1)
features_ham['#unique_links'] = features_ham.links.apply(lambda x: len(x.split(',')))

# #Count of number of unique links in body (counting the host)
# features_ham['#unique_links'] = features_ham.link_df.apply(lambda x: len(x.split(',')))
# features_ham['#unique_links']


In [82]:
#body of email to words 
import re
import nltk
#nltk.download()
from nltk.corpus import stopwords
stop  = stopwords.words("english")

In [83]:
def regex_normalize(text_body):
    normal_text = re.sub("[^a-zA-Z]", " ", text_body)
    return normal_text

def regex_lower(clean_body):
    normal_text_2 = clean_body.lower().split()
    return normal_text_2

def remove_short_words(clean_me):
    text1 = re.sub(r'\b\w{1,2}\b', '',  clean_me)
    return text1

In [84]:
#only words
features_ham['body_text_normalize'] = features_ham.body.apply(regex_normalize)

# #remove short words less than 3 characters
features_ham['body_text_normalize'] = features_ham.body.apply(remove_short_words)
# #lower
features_ham['body_text_normalize'] = features_ham.body_text_normalize.apply(regex_lower)
features_ham['body_text_normalize'] = features_ham['body_text_normalize'].apply(lambda x: [item for item in x if item not in stop])


In [86]:
#bag of words for subject
REGEX = re.compile(r"[^a-zA-Z]")
def tokenize(text):
    return [tok.strip().lower() for tok in REGEX.split(text)]

features_ham['subject'] = features_ham['subject'].apply(tokenize)



In [91]:
#convert list to string in dataframe
features_ham['subject_string'] = [','.join(map(str, l)) for l in features_ham['subject']]

In [92]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(tokenizer=tokenize, min_df=0, stop_words="english", max_features=200)
counts = cv.fit_transform(features_ham.subject_string.values).toarray().ravel()
words = np.array(cv.get_feature_names())
#normalize
counts = counts / float(counts.max())

/usr/local/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
words

In [93]:
#sqlalchemy 
from sqlalchemy import create_engine
#2.-Turn on database engine
engine = create_engine('sqlite:///ham_emails.db', echo=True)

#dbEngine=sqlalchemy.create_engine('sqlite:////home/stephen/db1.db') # ensure this is the correct path for the sqlite file. 

#I also want to add a new table from a dataframe in sqlite (a small one) 
 
newdata.to_sql(name = 'TUmain01012012',con= engine, index=False, if_exists='replace')

2022-08-13 20:05:02,161 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("TUmain01012012")
2022-08-13 20:05:02,162 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-08-13 20:05:02,165 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("TUmain01012012")
2022-08-13 20:05:02,166 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-08-13 20:05:02,169 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2022-08-13 20:05:02,169 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-08-13 20:05:02,171 INFO sqlalchemy.engine.Engine PRAGMA main.table_xinfo("TUmain01012012")
2022-08-13 20:05:02,172 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-08-13 20:05:02,178 INFO sqlalchemy.engine.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = ? AND type = 'table'
2022-08-13 20:05:02,178 INFO sqlalchemy.engine.Engine [raw sql] ('TUmain01012012',)
2022-08-13 20:05:02,181 INFO sqlalchemy.engine.Engine PRAGMA mai

99

In [94]:
features_ham.columns

Index(['delivered-to', 'received', 'domainkey-status', 'received-spf',
       'x-gmail-fetch-info', 'return-path', 'authentication-results', 'x-sbrs',
       'dkim-signature', 'domainkey-signature', 'date', 'from', 'reply-to',
       'to', 'message-id', 'subject', 'mime-version', 'content-type', 'x-mid',
       'x-orgid', 'list-unsubscribe', 'body_part', 'body',
       'content-transfer-encoding', 'x-originating-ip', 'x-mailer', 'sender',
       'x-linkedin-template', 'x-linkedin-class', 'x-linkedin-fbl',
       'x-mailer-lid', 'x-mailer-recptid', 'x-mailer-sid', 'x-mailer-sent-by',
       'content-disposition', 'x-antiabuse', 'x-dkim', 'x-sgxh1', 'x-rext',
       'x-cid', 'x-priority', 'errors-to', 'x-list-unsubscribe',
       'x-unsubscribe-web', 'x-icpinfo', 'x-return-path-hint', 'x-eid',
       'x-pid', 'x-virtualserver', 'x-mailingid', 'x-smheadermap',
       'x-destination-id', 'x-smfbl', 'x-virtualservergroup', 'guid',
       'x-indid', 'x-vendor', 'x-campaignid', 'x-mailer-vers

In [100]:
features_final = features_ham[['from', 'subject', 'date', 'delivered-to', 
'received', 'received-spf', 'reply-to', 'return-path', 
'feature_first_time_from', 'list-unsubscribe', 
'subject_string', 'ip_of_sender', 
'body', 'body_part', 'body_text_normalize', 'subject_string', 
'#unique_links']]

In [101]:
features_final = features_final.loc[:,~features_final.columns.duplicated()].copy()

In [102]:
features_final.columns

Index(['from', 'subject', 'date', 'delivered-to', 'received', 'received-spf',
       'reply-to', 'return-path', 'feature_first_time_from',
       'list-unsubscribe', 'subject_string', 'ip_of_sender', 'body',
       'body_part', 'body_text_normalize', '#unique_links'],
      dtype='object')

In [103]:
import numpy as np
import pandas as pd
from pandas_profiling import ProfileReport
profile = ProfileReport(features_final, title="Pandas Profiling Report")
profile

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
#ETL cleanse check for missing or empty values 
import missingno as msno
msno.bar(features_final)

In [99]:
features_ham['date'] = [','.join(map(str, l)) for l in features_ham['date']]
features_ham['date']

0    [,',S,u,n,,, ,1, ,J,a,n, ,2,0,1,2, ,0,1,:,4,1,...
0    [,',M,o,n,,, ,2, ,J,a,n, ,2,0,1,2, ,1,2,:,2,1,...
0    [,',T,h,u,,, ,0,5, ,J,a,n, ,2,0,1,2, ,0,1,:,3,...
0    [,',F,r,i,,, ,6, ,J,a,n, ,2,0,1,2, ,0,8,:,0,7,...
0    [,',S,u,n,,, ,0,8, ,J,a,n, ,2,0,1,2, ,0,1,:,3,...
                           ...                        
0    [,',S,u,n,,, , ,8, ,A,p,r, ,2,0,1,2, ,0,7,:,1,...
0    [,',M,o,n,,, ,9, ,A,p,r, ,2,0,1,2, ,1,1,:,1,4,...
0    [,',M,o,n,,, ,9, ,A,p,r, ,2,0,1,2, ,2,0,:,2,3,...
0    [,',T,h,u,,, ,1,2, ,A,p,r, ,2,0,1,2, ,0,4,:,2,...
0    [,',T,h,u,,, ,1,2, ,A,p,r, ,2,0,1,2, ,1,2,:,2,...
Name: date, Length: 99, dtype: object

In [13]:
remove = []
cols = df.columns
for i in range(len(cols)-1):
    v = df[cols[i]].values
    for j in range(i+1,len(cols)):
        if np.array_equal(v,df[cols[j]].values):
            remove.append(cols[j])

output.drop(remove, axis=1, inplace=True)
output.columns

NameError: name 'output' is not defined

In [ ]:
# Create a cursor
cur = con.cursor()

In [ ]:
cur.execute("SELECT DISTINCT subject, count(*) as Counter FROM \
TUmain01012012 GROUP BY subject ORDER BY Counter ASC;")
cur.fetchall()

In [ ]:
#read from sql and convert to pandas 
df_sql = pd.read_sql("select * from TUmain01012012", con)
df_sql

In [ ]:
pd.set_option("display.max_rows", None)

In [ ]:
df_sql.body.head(100)

In [ ]:
import numpy as np
remove = []
cols = df.columns
for i in range(len(cols)-1):
    v = df[cols[i]].values
    for j in range(i+1,len(cols)):
        if np.array_equal(v,df[cols[j]].values):
            remove.append(cols[j])

df.drop(remove, axis=1, inplace=True)